## Manually label and upload photos to the database

* **Input:** folder with classifc image classification format:

```
images/
    class_1/
        image_1.jpg
        image_2.jpg
        ...
    class_2/
        image_3.jpeg
        image_4.jpeg
        ...
    ...
```

* **Output:** Labelled images stored in GCP and tracked with Weights & Biases Artifacts. 


In [88]:
# Append the upper level directory to sys
import sys
sys.path.append("..")

import pandas as pd
import numpy as np

from pathlib import Path

In [89]:
# !rm -rf food_photos/*
# !rm -rf _MACOSX
# !rm -rf _MACOSX/*

In [90]:
# !unzip -q 2023-02-15-food_photos.zip

In [91]:
# Get list of all paths in food_photos directory
target_dir = '2023-03-04-food_photos'
path = Path(target_dir)
all_paths = sorted(list(path.glob('*/*.jp*g')))
all_paths[:10]

[PosixPath('2023-03-04-food_photos/almonds/004b2ffe-bf33-4694-a14e-bb1be127b69f.jpeg'),
 PosixPath('2023-03-04-food_photos/almonds/01cfd148-9f25-4f9c-a2ad-a613db055536.jpeg'),
 PosixPath('2023-03-04-food_photos/almonds/022bb17d-685c-4551-b4c2-2dafddc8ac31.jpeg'),
 PosixPath('2023-03-04-food_photos/almonds/0278f758-ba9b-43c1-ad93-55d13277f783.jpeg'),
 PosixPath('2023-03-04-food_photos/almonds/0439a887-1f2d-4f8b-93a7-2dacdc821da4.jpeg'),
 PosixPath('2023-03-04-food_photos/almonds/0464e873-1d7f-4bb4-8d39-995241264b58.jpeg'),
 PosixPath('2023-03-04-food_photos/almonds/06747748-46c8-43fc-8fca-4b8d70fd1c12.jpeg'),
 PosixPath('2023-03-04-food_photos/almonds/08cf5fda-5d7d-4d13-8dec-8bd050734f3c.jpeg'),
 PosixPath('2023-03-04-food_photos/almonds/0b69fa2f-f9e3-4e04-a11c-5289d56a0d28.jpeg'),
 PosixPath('2023-03-04-food_photos/almonds/0d38fd02-11e6-47bc-bfcf-0bd4fecd7e2c.jpeg')]

In [92]:
len(all_paths)

900

In [93]:
import uuid

def is_valid_uuid(string):
    try:
        uuid.UUID(string)
        return True
    except ValueError:
        return False

filename = "some-file-name"
if is_valid_uuid(filename):
    print(f"{filename} is a valid UUID")
else:
    print(f"{filename} is not a valid UUID")

some-file-name is not a valid UUID


In [94]:
# Loop through all_paths, if the path is a UUID, pass, if not, rename the file to a UUID
for path in all_paths:
    if not is_valid_uuid(path.stem):
        print(f"[INFO] {path} isn't valud UUID, renaming to UUID...")
        new_name = f"{uuid.uuid4()}.jpeg"
        new_path_name = path.parent / new_name
        path.rename(new_path_name)
        print(f"[INFO] Renamed {path} to {new_path_name}")

# Get list of all paths in food_photos directory
path = Path(target_dir)
all_paths = sorted(list(path.glob('*/*.jpeg')))
all_paths[:10]

[PosixPath('2023-03-04-food_photos/almonds/004b2ffe-bf33-4694-a14e-bb1be127b69f.jpeg'),
 PosixPath('2023-03-04-food_photos/almonds/01cfd148-9f25-4f9c-a2ad-a613db055536.jpeg'),
 PosixPath('2023-03-04-food_photos/almonds/022bb17d-685c-4551-b4c2-2dafddc8ac31.jpeg'),
 PosixPath('2023-03-04-food_photos/almonds/0278f758-ba9b-43c1-ad93-55d13277f783.jpeg'),
 PosixPath('2023-03-04-food_photos/almonds/0439a887-1f2d-4f8b-93a7-2dacdc821da4.jpeg'),
 PosixPath('2023-03-04-food_photos/almonds/0464e873-1d7f-4bb4-8d39-995241264b58.jpeg'),
 PosixPath('2023-03-04-food_photos/almonds/06747748-46c8-43fc-8fca-4b8d70fd1c12.jpeg'),
 PosixPath('2023-03-04-food_photos/almonds/08cf5fda-5d7d-4d13-8dec-8bd050734f3c.jpeg'),
 PosixPath('2023-03-04-food_photos/almonds/0b69fa2f-f9e3-4e04-a11c-5289d56a0d28.jpeg'),
 PosixPath('2023-03-04-food_photos/almonds/0d38fd02-11e6-47bc-bfcf-0bd4fecd7e2c.jpeg')]

In [95]:
# Find the number of .jpeg images in each subfolder of food_photos and create a dictionary mapping the subfolder name to the number of images
subfolder_to_num_images = {}
for p in all_paths:
    subfolder = p.parent.stem
    if subfolder in subfolder_to_num_images:
        subfolder_to_num_images[subfolder] += 1
    else:
        subfolder_to_num_images[subfolder] = 1
subfolder_to_num_images

{'almonds': 159,
 'cashews': 177,
 'macadamia': 5,
 'peanuts': 4,
 'pecans': 214,
 'pistachios': 158,
 'walnuts': 183}

In [96]:
len(subfolder_to_num_images)

7

In [97]:
# Get all food_types from all_paths
food_types = sorted(list(set([p.parent.name for p in all_paths])))
food_types

['almonds',
 'cashews',
 'macadamia',
 'peanuts',
 'pecans',
 'pistachios',
 'walnuts']

## Download original labels from Weights & Biases

In [98]:
# Get config
from configs.default_config import config

args = config
print(args)

namespace(annotations_columns_to_export=['filename', 'image_name', 'class_name', 'label', 'split', 'clear_or_confusing', 'whole_food_or_dish', 'one_food_or_multiple', 'label_last_updated_at', 'label_source', 'image_source'], auto_augment=True, batch_size=128, epochs=10, gs_bucket_name='food_vision_bucket_with_object_versioning', gs_image_storage_path='https://storage.cloud.google.com/food_vision_bucket_with_object_versioning/all_images/', input_size=224, label_smoothing=0.1, learning_rate=0.001, model='coatnext_nano_rw_224', num_to_try_and_autocorrect=1000, num_top_n_preds=5, path_to_gcp_credentials='utils/google-storage-key.json', path_to_label_studio_api_key='utils/label_studio_api_key.json', pretrained=True, seed=42, use_mixed_precision=True, wandb_dataset_artifact='food_vision_199_classes_images:latest', wandb_job_type='', wandb_labels_artifact='food_vision_labels:latest', wandb_model_artifact='trained_model:latest', wandb_project='test_wandb_artifacts_by_reference', wandb_run_note

In [99]:
# Connect to GCP
from utils.gcp_utils import set_gcp_credentials, test_gcp_connection
set_gcp_credentials(path_to_key="../utils/google-storage-key.json")
test_gcp_connection()

[INFO] GCP credentials set!
[INFO] GCP connection successful! Access to GCP for saving/loading data and models available.


In [100]:
import wandb

# Initialize a new run
from utils.wandb_utils import wandb_load_artifact, wandb_download_and_load_labels

notes = f"add {len(all_paths)} manually taken photos to the training dataset"

run = wandb.init(project=args.wandb_project, 
                 job_type=args.wandb_job_type,
                 tags=['manual_photo_upload'],
                 notes=notes)

annotations, class_names, class_dict, reverse_class_dict, labels_path = wandb_download_and_load_labels(wandb_run=run,
wandb_labels_artifact_name=args.wandb_labels_artifact)

[INFO] Labels directory: ./artifacts/food_vision_labels:v31
[INFO] Labels path: artifacts/food_vision_labels:v31/annotations.csv
[INFO] Working with: 245 classes


In [101]:
len(annotations.label.unique())

245

In [102]:
# Make a copy of the annotations
original_annotations = annotations.copy()

In [103]:
# Get the columns of the annotations
columns_to_create = list(original_annotations.columns)
columns_to_create

['filename',
 'image_name',
 'class_name',
 'label',
 'split',
 'clear_or_confusing',
 'whole_food_or_dish',
 'one_food_or_multiple',
 'label_last_updated_at',
 'label_source',
 'image_source']

In [104]:
# Get all the image paths from food_photos
image_paths = list(Path(target_dir).glob('*/*.jpeg'))
len(image_paths)

900

In [105]:
# Check to see what food_types are in class_names
food_types_in_class_names = [food_type for food_type in food_types if food_type in class_names]

# Check to see what food_types are not in class_names
food_types_not_in_class_names = [food_type for food_type in food_types if food_type not in class_names]
len(food_types_in_class_names)

6

In [106]:
food_types_in_class_names

['almonds', 'cashews', 'macadamia', 'pecans', 'pistachios', 'walnuts']

In [107]:
food_types_not_in_class_names

['peanuts']

In [108]:
# Check to see if "macadamia" in class_names
[class_name for class_name in class_names if "pis" in class_name]

['pistachio', 'pistachios']

In [109]:
# How to slot new classes into existing class_dict?
# Then update the existing annotations with the new class labels (e.g. "apple_green" -> 2, "apple_red" -> 3)

# Add new classes to class_dict
for food_type in food_types_not_in_class_names:
    class_dict[len(class_dict)] = food_type
    class_names.append(food_type)
    reverse_class_dict[food_type] = len(class_dict)-1

In [110]:
len(class_dict)

246

In [111]:
from utils.misc import get_now_time

# Create a list of dictionaries and fill out the columns of the annotations
columns = ['filename',
 'image_name',
 'class_name',
 'label',
 'split',
 'clear_or_confusing',
 'whole_food_or_dish',
 'one_food_or_multiple',
 'label_last_updated_at',
 'label_source',
 'image_source']

food_image_dict_list = []

LABEL_SOURCE = "manual_upload"
IMAGE_SOURCE = "manual_upload"
LABEL_LAST_UPDATED_AT = get_now_time()

for image_path in image_paths:
    food_image_dict = {}
    food_image_dict['filename'] = image_path
    food_image_dict['image_name'] = image_path.name
    food_image_dict['class_name'] = image_path.parent.name
    food_image_dict['label'] = reverse_class_dict[image_path.parent.name]
    # food_image_dict['split'] = 'train'
    # Label 20% of the images as test
    if np.random.random() < 0.2:
        food_image_dict['split'] = 'test'
    else:
        food_image_dict['split'] = 'train'
    food_image_dict['clear_or_confusing'] = 'clear'
    food_image_dict['whole_food_or_dish'] = 'whole_food'
    food_image_dict['one_food_or_multiple'] = 'one_food'
    food_image_dict['label_last_updated_at'] = LABEL_LAST_UPDATED_AT
    food_image_dict['label_source'] = LABEL_SOURCE
    food_image_dict['image_source'] = IMAGE_SOURCE

    food_image_dict_list.append(food_image_dict)

# Create a dataframe from the list of dictionaries
import pandas as pd
new_annotations = pd.DataFrame(food_image_dict_list)
new_annotations.head()

,filename,image_name,class_name,label,split,clear_or_confusing,whole_food_or_dish,one_food_or_multiple,label_last_updated_at,label_source,image_source
0,2023-03-04-food_photos/macadamia/d041fc31-5d69...,d041fc31-5d69-499a-a697-572cbd5b8575.jpeg,macadamia,124,train,clear,whole_food,one_food,2023-03-03_21-33-46,manual_upload,manual_upload
1,2023-03-04-food_photos/macadamia/af18c02a-171b...,af18c02a-171b-4b52-8f36-9c0f54972e1e.jpeg,macadamia,124,test,clear,whole_food,one_food,2023-03-03_21-33-46,manual_upload,manual_upload
2,2023-03-04-food_photos/macadamia/9d20fbb3-f7f4...,9d20fbb3-f7f4-4a14-999b-24a19103c008.jpeg,macadamia,124,train,clear,whole_food,one_food,2023-03-03_21-33-46,manual_upload,manual_upload
3,2023-03-04-food_photos/macadamia/78b08628-75e2...,78b08628-75e2-4715-ba46-570eaeb1dc3d.jpeg,macadamia,124,train,clear,whole_food,one_food,2023-03-03_21-33-46,manual_upload,manual_upload
4,2023-03-04-food_photos/macadamia/da455e2a-40d5...,da455e2a-40d5-430b-bfed-13d51335db2a.jpeg,macadamia,124,train,clear,whole_food,one_food,2023-03-03_21-33-46,manual_upload,manual_upload


In [112]:
# Print how many images are train/test in new_annotations
new_annotations['split'].value_counts()

train    719
test     181
Name: split, dtype: int64

In [113]:
# Find a random sample of 10 images where the label value is 199 or higher
new_annotations[new_annotations['label'] >= 199].sample(10)

,filename,image_name,class_name,label,split,clear_or_confusing,whole_food_or_dish,one_food_or_multiple,label_last_updated_at,label_source,image_source
277,2023-03-04-food_photos/walnuts/c2368180-a6a2-4...,c2368180-a6a2-4b13-a722-cacc89214ad7.jpeg,walnuts,237,test,clear,whole_food,one_food,2023-03-03_21-33-46,manual_upload,manual_upload
315,2023-03-04-food_photos/walnuts/9a9803d5-3a29-4...,9a9803d5-3a29-4b79-851a-f5bf5cb10d13.jpeg,walnuts,237,train,clear,whole_food,one_food,2023-03-03_21-33-46,manual_upload,manual_upload
226,2023-03-04-food_photos/walnuts/601ca544-9500-4...,601ca544-9500-4ba2-b53a-26fddd60ee46.jpeg,walnuts,237,train,clear,whole_food,one_food,2023-03-03_21-33-46,manual_upload,manual_upload
248,2023-03-04-food_photos/walnuts/8d5af2f7-1f80-4...,8d5af2f7-1f80-4b64-87d5-a8d816e22ac7.jpeg,walnuts,237,train,clear,whole_food,one_food,2023-03-03_21-33-46,manual_upload,manual_upload
234,2023-03-04-food_photos/walnuts/c378e03e-a954-4...,c378e03e-a954-4744-baea-25d810945dc5.jpeg,walnuts,237,train,clear,whole_food,one_food,2023-03-03_21-33-46,manual_upload,manual_upload
300,2023-03-04-food_photos/walnuts/174d135d-809f-4...,174d135d-809f-40fc-8101-12d33e02bec6.jpeg,walnuts,237,test,clear,whole_food,one_food,2023-03-03_21-33-46,manual_upload,manual_upload
211,2023-03-04-food_photos/walnuts/af31bbcb-c62f-4...,af31bbcb-c62f-4d45-be2a-4fb3328ceb23.jpeg,walnuts,237,train,clear,whole_food,one_food,2023-03-03_21-33-46,manual_upload,manual_upload
254,2023-03-04-food_photos/walnuts/92091f26-130a-4...,92091f26-130a-48d9-9c99-11bbc86ae93a.jpeg,walnuts,237,train,clear,whole_food,one_food,2023-03-03_21-33-46,manual_upload,manual_upload
346,2023-03-04-food_photos/walnuts/192894c5-8fdb-4...,192894c5-8fdb-4d5b-b285-c57797115dfa.jpeg,walnuts,237,train,clear,whole_food,one_food,2023-03-03_21-33-46,manual_upload,manual_upload
313,2023-03-04-food_photos/walnuts/aee4aa48-2fc1-4...,aee4aa48-2fc1-4c4e-8c80-6482bd27b343.jpeg,walnuts,237,train,clear,whole_food,one_food,2023-03-03_21-33-46,manual_upload,manual_upload


In [114]:
len(original_annotations), len(new_annotations)

(110276, 900)

In [115]:
# Append the new_annotations to the original_annotations
updated_annotations = pd.concat([original_annotations, new_annotations], ignore_index=True)

# updated_annotations = original_annotations.(new_annotations, ignore_index=True)
updated_annotations

,filename,image_name,class_name,label,split,clear_or_confusing,whole_food_or_dish,one_food_or_multiple,label_last_updated_at,label_source,image_source
0,test/pain_au_chocolat/4fd7cb42-bd7f-48f1-bfdc-...,4fd7cb42-bd7f-48f1-bfdc-607c2f54b788.jpg,pain_au_chocolat,149,test,NaN,NaN,NaN,NaN,NaN,internet_download
1,test/pain_au_chocolat/2062f52a-781c-4e4f-b8a7-...,2062f52a-781c-4e4f-b8a7-0a108934f453.jpg,pain_au_chocolat,149,test,NaN,NaN,NaN,NaN,NaN,internet_download
2,test/pain_au_chocolat/8003e0f6-37e8-460d-9c14-...,8003e0f6-37e8-460d-9c14-e7c6fe44a37f.jpg,pain_au_chocolat,149,test,NaN,NaN,NaN,NaN,NaN,internet_download
3,test/pain_au_chocolat/839437c8-c643-408f-9f04-...,839437c8-c643-408f-9f04-d0d3bec238c3.jpg,pain_au_chocolat,149,test,NaN,NaN,NaN,NaN,NaN,internet_download
4,test/pain_au_chocolat/ca5c13ff-a535-4b69-9144-...,ca5c13ff-a535-4b69-9144-e06275e01e35.jpg,pain_au_chocolat,149,test,NaN,NaN,NaN,NaN,NaN,internet_download
...,...,...,...,...,...,...,...,...,...,...,...
111171,2023-03-04-food_photos/pistachios/03b6bef7-bd8...,03b6bef7-bd8d-455a-aa75-7ec8899846d7.jpeg,pistachios,166,train,clear,whole_food,one_food,2023-03-03_21-33-46,manual_upload,manual_upload
111172,2023-03-04-food_photos/pistachios/37f17b35-d40...,37f17b35-d404-4eb5-af31-07d9fd3ae1ef.jpeg,pistachios,166,train,clear,whole_food,one_food,2023-03-03_21-33-46,manual_upload,manual_upload
111173,2023-03-04-food_photos/pistachios/7455682e-b74...,7455682e-b746-4de7-bf86-2d2caa3d9a05.jpeg,pistachios,166,test,clear,whole_food,one_food,2023-03-03_21-33-46,manual_upload,manual_upload
111174,2023-03-04-food_photos/pistachios/46f386e9-e90...,46f386e9-e903-4bf8-a8df-0fb0f72431cb.jpeg,pistachios,166,train,clear,whole_food,one_food,2023-03-03_21-33-46,manual_upload,manual_upload


In [116]:
# How many unique class_names are in original_annotations?
len(original_annotations['class_name'].unique())

245

In [117]:
# How many unique class_names are in updated_annotations?
len(updated_annotations['class_name'].unique())

246

In [118]:
from typing import List, Dict, Tuple

def get_updated_class_names_class_dict_and_reverse_class_dict(df: pd.DataFrame) -> Tuple[List[str], Dict[int, str], Dict[str, int]]:
    """Get an updated class names list, class dict and reverse class dict from a dataframe."""
    updated_class_names = sorted(df.class_name.unique())
    updated_class_dict = {i: class_name for i, class_name in enumerate(updated_class_names)}
    updated_reverse_class_dict = {class_name: i for i, class_name in enumerate(updated_class_names)}
    assert len(updated_class_dict) == len(updated_reverse_class_dict), "Class dict and reverse class dict are not the same length"
    return updated_class_names, updated_class_dict, updated_reverse_class_dict

def map_updated_class_dict_to_updated_annotations(df: pd.DataFrame, updated_reverse_class_dict: Dict[str, int]) -> pd.DataFrame:
    """Map updated class dict to updated annotations dataframe.
    
    For example, go from {"apple_red": 1} -> df["label"] = 1
    """
    updated_annotations = df.copy()
    updated_annotations.loc[:, 'label'] = updated_annotations['class_name'].map(updated_reverse_class_dict)
    assert len(updated_annotations.label.unique()) == len(updated_reverse_class_dict), "Number of unique labels in updated annotations does not match number of unique class names in updated reverse class dict"
    return updated_annotations

In [119]:
updated_class_names, updated_class_dict, updated_reverse_class_dict = get_updated_class_names_class_dict_and_reverse_class_dict(updated_annotations)

updated_annotations = map_updated_class_dict_to_updated_annotations(updated_annotations, updated_reverse_class_dict)

len(updated_class_names), updated_class_names[:10]

(246,
 ['almond_butter',
  'almonds',
  'apple_green',
  'apple_red',
  'apricot',
  'asparagus',
  'avocado',
  'bacon',
  'bacon_and_egg_burger',
  'bagel'])

In [120]:
# Check the difference in lengths between the original_annotations and updated_annotations
num_differences = len(updated_annotations) - len(original_annotations)
num_differences

900

In [121]:
# Upload the updated annotations to Google Storage and track the changes
from utils.gcp_utils import upload_to_gs, rename_blob, delete_blob
from utils.wandb_utils import wandb_add_artifact_with_reference
from utils.misc import get_now_time
import os

GS_BUCKET_NAME = config.gs_bucket_name

UPDATED_ANNOTATIONS_TARGET_FILENAME = "updated_annotations.csv"
ORIGINAL_ANNOTATIONS_TARGET_FILENAME = "annotations.csv"

# Export the updated annotations to a CSV
columns_to_export = config.annotations_columns_to_export
print(f"[INFO] Exporting the following columns to {UPDATED_ANNOTATIONS_TARGET_FILENAME}: {columns_to_export}")

# TODO: Check if the updated_annotations_reset_index and the original_annotations actually differ, if so save them and upload them, else exit
if num_differences > 0:
    print(f"[INFO] {num_differences} changes to annotations.csv, updated label files and original annotations are different, saving the updated annotations.csv")

    # Export the updated_annotations_reset_index to a csv
    updated_annotations[columns_to_export].to_csv(UPDATED_ANNOTATIONS_TARGET_FILENAME, index=False)

    # Upload the updated CSV to Google Storage
    upload_to_gs(bucket_name=GS_BUCKET_NAME, 
                 source_file_name=UPDATED_ANNOTATIONS_TARGET_FILENAME, 
                 destination_blob_name=UPDATED_ANNOTATIONS_TARGET_FILENAME)

    # Rename the old CSV on Google Storage
    bucket_to_move_old_annotations_to = "old_annotations"
    name_to_rename_old_annotations = os.path.join(bucket_to_move_old_annotations_to, f"{get_now_time()}_old_annotations.csv")

    rename_blob(bucket_name=GS_BUCKET_NAME,
                blob_name=ORIGINAL_ANNOTATIONS_TARGET_FILENAME,
                new_name=name_to_rename_old_annotations)

    # Rename the "updated_annotations.csv" on Google Storage to "annotations.csv" 
    rename_blob(bucket_name=GS_BUCKET_NAME,
                blob_name=UPDATED_ANNOTATIONS_TARGET_FILENAME,
                new_name=ORIGINAL_ANNOTATIONS_TARGET_FILENAME)

    # Track the changes in the annotations with Weights & Biases
    annotations_path_on_gcs = f"gs://{GS_BUCKET_NAME}/{ORIGINAL_ANNOTATIONS_TARGET_FILENAME}"
    wandb_add_artifact_with_reference(wandb_run=run,
                                      artifact_name="food_vision_labels",
                                      artifact_type="labels",
                                      description="Labels for FoodVision project",
                                      reference_path=annotations_path_on_gcs)
else:
    print("[INFO] No changes to annotations.csv, updated label files and original annotations are the same, try fixing/updating the label files and try again")

[INFO] Exporting the following columns to updated_annotations.csv: ['filename', 'image_name', 'class_name', 'label', 'split', 'clear_or_confusing', 'whole_food_or_dish', 'one_food_or_multiple', 'label_last_updated_at', 'label_source', 'image_source']
[INFO] 900 changes to annotations.csv, updated label files and original annotations are different, saving the updated annotations.csv
[INFO] Uploading updated_annotations.csv to updated_annotations.csv...
[INFO] Connected to Google Storage bucket: food_vision_bucket_with_object_versioning
[INFO] File updated_annotations.csv uploaded to food_vision_bucket_with_object_versioning/updated_annotations.csv.
[INFO] File size: 21333560 bytes
[INFO] Blob annotations.csv has been renamed to old_annotations/2023-03-03_21-34-14_old_annotations.csv
[INFO] Blob updated_annotations.csv has been renamed to annotations.csv
[INFO] Logging 'food_vision_labels' from 'gs://food_vision_bucket_with_object_versioning/annotations.csv' to Weights & Biases...


## Upload all photos in food_photos to Google Storage

In [122]:
from utils.gcp_utils import upload_to_gs, get_list_of_blobs

# Get list of images already in GCP bucket
gs_image_paths = get_list_of_blobs(bucket_name=GS_BUCKET_NAME, prefix="all_images")
print(f"[INFO] There are {len(gs_image_paths)} images in the GCP bucket")

[INFO] There are 109409 images in the GCP bucket


In [123]:
gs_image_paths_name = [str(blob.name).split("/")[-1] for blob in gs_image_paths]
gs_image_paths_name[:10]

['000226a7-5332-4f45-b0e9-6760e9bd6d3e.jpeg',
 '0003c8a1-7f64-4540-9256-3252f0981035.jpeg',
 '00045a69-b09f-4293-8c2e-a7ba27964fb6.jpg',
 '0004a23a-88b3-4aae-a1b0-c9ebe77d31b8.jpeg',
 '00065302-c0e7-4634-ab63-5ddd16bfdeb8.jpeg',
 '0008eda8-cd90-4394-bdac-aa1574f7738d.jpeg',
 '0009bea1-1577-4db5-bc60-b27eaed2d276.jpg',
 '000b5cca-0b52-452b-9a11-12c44bfaab12.jpeg',
 '000b9348-2afc-425d-91a8-765c04483a41.jpeg',
 '000bd61c-f25c-415c-8cf1-3adc2d788320.jpg']

In [124]:
len(gs_image_paths_name)

109409

In [125]:
print(target_dir)

2023-03-04-food_photos


In [126]:
# Upload the images to Google Storage
!gsutil -m cp -r $target_dir/*/*.jp*g gs://$GS_BUCKET_NAME/all_images/

Copying file://2023-03-04-food_photos/almonds/004b2ffe-bf33-4694-a14e-bb1be127b69f.jpeg [Content-Type=image/jpeg]...
Copying file://2023-03-04-food_photos/almonds/01cfd148-9f25-4f9c-a2ad-a613db055536.jpeg [Content-Type=image/jpeg]...
Copying file://2023-03-04-food_photos/almonds/022bb17d-685c-4551-b4c2-2dafddc8ac31.jpeg [Content-Type=image/jpeg]...
Copying file://2023-03-04-food_photos/almonds/0278f758-ba9b-43c1-ad93-55d13277f783.jpeg [Content-Type=image/jpeg]...
Copying file://2023-03-04-food_photos/almonds/0439a887-1f2d-4f8b-93a7-2dacdc821da4.jpeg [Content-Type=image/jpeg]...
Copying file://2023-03-04-food_photos/almonds/0464e873-1d7f-4bb4-8d39-995241264b58.jpeg [Content-Type=image/jpeg]...
Copying file://2023-03-04-food_photos/almonds/06747748-46c8-43fc-8fca-4b8d70fd1c12.jpeg [Content-Type=image/jpeg]...
Copying file://2023-03-04-food_photos/almonds/08cf5fda-5d7d-4d13-8dec-8bd050734f3c.jpeg [Content-Type=image/jpeg]...
Copying file://2023-03-04-food_photos/almonds/0e568230-c8bf-432b

In [135]:
# # Test uploading of 1 image
# one_image = image_paths[0]

# # Upload the image to Google Storage
# upload_to_gs(bucket_name=GS_BUCKET_NAME,
#              source_file_name=one_image,
#              destination_blob_name=f"all_images/{one_image.name}")

In [127]:
# Get list of images already in GCP bucket
gs_image_paths = get_list_of_blobs(bucket_name=GS_BUCKET_NAME, prefix="all_images")
print(f"[INFO] There are {len(gs_image_paths)} images in the GCP bucket")

[INFO] There are 110309 images in the GCP bucket


In [128]:
gs_image_paths_name = [str(blob.name).split("/")[-1] for blob in gs_image_paths]
gs_image_paths_name[:10]

['000226a7-5332-4f45-b0e9-6760e9bd6d3e.jpeg',
 '0003c8a1-7f64-4540-9256-3252f0981035.jpeg',
 '00045a69-b09f-4293-8c2e-a7ba27964fb6.jpg',
 '0004a23a-88b3-4aae-a1b0-c9ebe77d31b8.jpeg',
 '00065302-c0e7-4634-ab63-5ddd16bfdeb8.jpeg',
 '0008eda8-cd90-4394-bdac-aa1574f7738d.jpeg',
 '0009bea1-1577-4db5-bc60-b27eaed2d276.jpg',
 '000b5cca-0b52-452b-9a11-12c44bfaab12.jpeg',
 '000b9348-2afc-425d-91a8-765c04483a41.jpeg',
 '000bd61c-f25c-415c-8cf1-3adc2d788320.jpg']

In [129]:
for image_name in image_paths:
    if "b322004c-d78e-47e9-a2c1-1d5d116b1601.jpeg" in image_name.name:
        print(f"[INFO] {image_name} in GCP bucket")
        # print(f"[INFO] {image_name} not in GCP bucket")

In [130]:
# Loop through image_paths and upload the image as long as its name isn't in gs_image_paths_name
from tqdm.auto import tqdm

num_images_uploaded = 0
for image_path in tqdm(image_paths):
    if image_path.name not in gs_image_paths_name:
        
        # Upload the image to Google Storage
        upload_to_gs(bucket_name=GS_BUCKET_NAME,
                     source_file_name=image_path,
                     destination_blob_name=f"all_images/{image_path.name}")
        
        num_images_uploaded += 1

print(f"[INFO] Uploaded {num_images_uploaded} images to GCP bucket")

  0%|          | 0/900 [00:00<?, ?it/s]

[INFO] Uploaded 0 images to GCP bucket


In [131]:
# Track the changes to GCP bucket with Weights & Biases
from utils.wandb_utils import wandb_add_artifact_with_reference

In [132]:
# Track updated images_dir in Weights & Biases
food_images_path_on_gs = "gs://food_vision_bucket_with_object_versioning/all_images"

wandb_add_artifact_with_reference(wandb_run=run,
                                  artifact_name="food_vision_199_classes_images",
                                  artifact_type="dataset",
                                  description="Images for FoodVision project",
                                  reference_path=food_images_path_on_gs)

[INFO] Logging 'food_vision_199_classes_images' from 'gs://food_vision_bucket_with_object_versioning/all_images' to Weights & Biases...


wandb: Generating checksum for up to 1000000000 objects with prefix "all_images"... Done. 23.6s


In [133]:
# Get list of images already in GCP bucket
gs_image_paths = get_list_of_blobs(bucket_name=GS_BUCKET_NAME, prefix="all_images")
print(f"[INFO] There are {len(gs_image_paths)} images in the GCP bucket")

[INFO] There are 110309 images in the GCP bucket


In [62]:
gs_image_paths[0].name

'all_images/000226a7-5332-4f45-b0e9-6760e9bd6d3e.jpeg'

In [ ]:
# Next:
# Create labels CSV for all photos in food_photos ✅
# Add labels CSV to original annotations ✅
# Upload all photos in food_photos to GCP ✅
# Track updates in W&B ✅

# Train a model and track how it performs (by only upgrading training data)